In [20]:
import sys
print(sys.executable)


c:\Users\Gur Levy\heinecan-4\.venv\Scripts\python.exe


In [21]:
!pip install requests


In [22]:
import requests
import csv
from datetime import datetime, timedelta


In [23]:
import requests
import csv
from datetime import datetime, timedelta

# api endpoint and params
API_KEY = 'd420319993e5906012bee411669261e0'
API_URL = 'https://api.openweathermap.org/data/3.0/onecall'

CITIES = {
    'monterrey': {'lat': 25.6866, 'lon': -100.3161},
    'amsterdam': {'lat': 52.3676, 'lon': 4.9041},
    'singapore': {'lat': 1.3521, 'lon': 103.8198}
}
#api request
def fetch_weather_data(city_name):
    if city_name not in CITIES:
        raise ValueError(f"City '{city_name}' is not supported.")
    
    coords = CITIES[city_name]
    params = {
        'lat': coords['lat'],
        'lon': coords['lon'],
        'exclude': 'current,minutely,daily,alerts',
        'appid': API_KEY,
        'units': 'metric'
    }

    response = requests.get(API_URL, params=params)
    response.raise_for_status()
    data = response.json()
    #timezone offset
    offset = data.get('timezone_offset', 0)
    hourly_data = data.get('hourly', [])

    csv_rows = []
    header = ['datetime', 'temp', 'feels_like', 'pressure', 'humidity', 'dew_point',
              'uvi', 'clouds', 'wind_speed', 'wind_deg', 'weather_description']
    #changing to datetime
    for entry in hourly_data:
        dt = datetime.utcfromtimestamp(entry['dt']) + timedelta(seconds=offset)
        weather_desc = entry.get('weather', [{}])[0].get('description', '')
        row = [
            dt.strftime("%Y-%m-%d %H:%M:%S"),
            entry.get("temp"),
            entry.get("feels_like"),
            entry.get("pressure"),
            entry.get("humidity"),
            entry.get("dew_point"),
            entry.get("uvi"),
            entry.get("clouds"),
            entry.get("wind_speed"),
            entry.get("wind_deg"),
            weather_desc
        ]
        csv_rows.append(row)
    #csv save
    csv_file = f'{city_name}_hourly_weather.csv'
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(csv_rows)

    print(f'File saved: {csv_file}')

fetch_weather_data("monterrey")
fetch_weather_data("amsterdam")
fetch_weather_data("singapore")

File saved: monterrey_hourly_weather.csv
File saved: amsterdam_hourly_weather.csv
File saved: singapore_hourly_weather.csv


C:\Users\Gur Levy\AppData\Local\Temp\ipykernel_19596\1580072142.py:40: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(entry['dt']) + timedelta(seconds=offset)


In [24]:
import requests
import csv
from datetime import datetime, timedelta
#location and api
API_KEY = 'd420319993e5906012bee411669261e0'
WEATHER_URL = 'https://api.openweathermap.org/data/3.0/onecall'
SOLAR_URL = 'https://api.openweathermap.org/energy/1.0/solar/data'

CITIES = {
    'monterrey': {'lat': 25.686613, 'lon': -100.316116},
    'amsterdam': {'lat': 52.3676, 'lon': 4.9041},
    'singapore': {'lat': 1.3521, 'lon': 103.8198}
}

def fetch_utci_inputs(city):
    if city not in CITIES:
        raise ValueError(f"City '{city}' is not supported.")
    
    lat = CITIES[city]['lat']
    lon = CITIES[city]['lon']

    weather_params = {
        'lat': lat,
        'lon': lon,
        'exclude': 'current,minutely,daily,alerts',
        'appid': API_KEY,
        'units': 'metric'
    }

    weather_response = requests.get(WEATHER_URL, params=weather_params)
    weather_data = weather_response.json()
    offset = weather_data.get('timezone_offset', 0)
# hourly weather from One Call API
    weather_by_hour = {}
    for entry in weather_data.get('hourly', []):
        dt = datetime.utcfromtimestamp(entry['dt']) + timedelta(seconds=offset)
        dt_str = dt.strftime("%Y-%m-%d %H")
        weather_by_hour[dt_str] = {
            'datetime': dt.strftime("%Y-%m-%d %H:%M:%S"),
            'temp': entry.get("temp"),
            'humidity': entry.get("humidity"),
            'wind_speed': entry.get("wind_speed")
        }

    # solar data 
    solar_by_hour = {}
    for i in range(7):
        date_str = (datetime.utcnow() + timedelta(days=i)).strftime('%Y-%m-%d')
        solar_params = {
            'lat': lat,
            'lon': lon,
            'date': date_str,
            'appid': API_KEY
        }

        solar_response = requests.get(SOLAR_URL, params=solar_params)
        if solar_response.status_code == 200:
            solar_data = solar_response.json()
            for h in solar_data.get('irradiance', {}).get('hourly', []):
                dt_str = f"{date_str} {h['hour']:02}"
                solar_by_hour[dt_str] = h['clear_sky']['ghi']
        else:
            print(f"[!] Solar API failed for {date_str} ({city}): {solar_response.status_code}")

    # merge and create final csv
    final_rows = []
    header = ['datetime', 'temp', 'humidity', 'wind_speed', 'solar_ghi']

    for dt_hour, weather in weather_by_hour.items():
        solar = solar_by_hour.get(dt_hour)
        if solar is not None:
            row = [
                weather['datetime'],
                weather['temp'],
                weather['humidity'],
                weather['wind_speed'],
                solar
            ]
            final_rows.append(row)

    output_file = f"{city}_utci_inputs.csv"
    with open(output_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(final_rows)

    print(f"UTCI input file saved for {city.title()}: {output_file}")

fetch_utci_inputs("monterrey")
fetch_utci_inputs("amsterdam")
fetch_utci_inputs("singapore")


C:\Users\Gur Levy\AppData\Local\Temp\ipykernel_19596\3070337885.py:36: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(entry['dt']) + timedelta(seconds=offset)
C:\Users\Gur Levy\AppData\Local\Temp\ipykernel_19596\3070337885.py:48: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  date_str = (datetime.utcnow() + timedelta(days=i)).strftime('%Y-%m-%d')


UTCI input file saved for Monterrey: monterrey_utci_inputs.csv
UTCI input file saved for Amsterdam: amsterdam_utci_inputs.csv
UTCI input file saved for Singapore: singapore_utci_inputs.csv


In [ ]:
import requests
import csv
from datetime import datetime, timedelta

API_KEY = 'd420319993e5906012bee411669261e0'
WEATHER_URL = 'https://api.openweathermap.org/data/3.0/onecall'
SOLAR_URL = 'https://api.openweathermap.org/energy/1.0/solar/data'

CITIES = {
    'monterrey': {'lat': 25.686613, 'lon': -100.316116},
    'amsterdam': {'lat': 52.3676, 'lon': 4.9041},
    'singapore': {'lat': 1.3521, 'lon': 103.8198}
}

def fetch_utci_inputs(city):
    if city not in CITIES:
        raise ValueError(f"City '{city}' not in list.")
    
    lat = CITIES[city]['lat']
    lon = CITIES[city]['lon']

    # --- Weather request ---
    weather_params = {
        'lat': lat,
        'lon': lon,
        'exclude': 'current,minutely,daily,alerts',
        'appid': API_KEY,
        'units': 'metric'
    }
    response = requests.get(WEATHER_URL, params=weather_params)
    response.raise_for_status()
    weather_data = response.json()

    offset = weather_data.get('timezone_offset', 0)
    weather_by_hour = {}

    for entry in weather_data.get('hourly', []):
        dt = datetime.utcfromtimestamp(entry['dt']) + timedelta(seconds=offset)
        dt_str = dt.strftime("%Y-%m-%d %H")
        weather_by_hour[dt_str] = {
            'datetime': dt.strftime("%Y-%m-%d %H:%M:%S"),
            'temp': entry.get("temp"),
            'humidity': entry.get("humidity"),
            'wind_speed': entry.get("wind_speed")
        }

    # --- Solar request ---
    solar_by_hour = {}
    for i in range(3):  # only 3 days needed to cover any gaps
        date_str = (datetime.utcnow() + timedelta(days=i)).strftime('%Y-%m-%d')
        solar_params = {
            'lat': lat,
            'lon': lon,
            'date': date_str,
            'appid': API_KEY
        }
        solar_resp = requests.get(SOLAR_URL, params=solar_params)
        if solar_resp.status_code == 200:
            solar_data = solar_resp.json()
            for h in solar_data.get('irradiance', {}).get('hourly', []):
                dt_str = f"{date_str} {h['hour']:02}"
                solar_by_hour[dt_str] = h['clear_sky']['ghi']
        else:
            print(f"[!] Solar API failed for {date_str}: {solar_resp.status_code}")

    # --- Merge ---
    final_rows = []
    header = ['datetime', 'temp', 'humidity', 'wind_speed', 'solar_ghi']

    for dt_hour, weather in weather_by_hour.items():
        solar = solar_by_hour.get(dt_hour)
        if solar is not None:
            final_rows.append([
                weather['datetime'],
                weather['temp'],
                weather['humidity'],
                weather['wind_speed'],
                solar
            ])

    # --- Save ---
    out_file = f"{city}_utci_inputs.csv"
    with open(out_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(final_rows)

    print(f" Saved: {out_file}")

# Run for all cities
for city in CITIES:
    fetch_utci_inputs(city)


C:\Users\Gur Levy\AppData\Local\Temp\ipykernel_19596\2850617802.py:38: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(entry['dt']) + timedelta(seconds=offset)
C:\Users\Gur Levy\AppData\Local\Temp\ipykernel_19596\2850617802.py:50: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  date_str = (datetime.utcnow() + timedelta(days=i)).strftime('%Y-%m-%d')


✅ Saved: monterrey_utci_inputs.csv
✅ Saved: amsterdam_utci_inputs.csv
✅ Saved: singapore_utci_inputs.csv


In [26]:
!pip install pandas

In [27]:
import pandas as pd
import numpy as np

def compute_utci(city):
    input_file = f"{city}_utci_inputs.csv"
    output_file = f"{city}_utci_results.csv"
    
    df = pd.read_csv(input_file)

    # Estimate MRT (Mean Radiant Temperature)
    df['mrt'] = df['temp'] + 0.03 * df['solar_ghi']

    # Water vapor pressure using Tetens formula (converted to kPa)
    es = 6.112 * np.exp((17.67 * df['temp']) / (df['temp'] + 243.5))  # hPa
    df['pa'] = (es * df['humidity'] / 100) / 10  # convert hPa to kPa

    # Empirical UTCI approximation (Bröde et al., 2012)
    df['utci'] = (
        0.6076 +
        0.9772 * df['temp'] +
        -2.2584 * df['wind_speed'] +
        0.3984 * (df['mrt'] - df['temp']) +
        5.1273 * df['pa']
    )

    df.to_csv(output_file, index=False)
    print(f"UTCI results saved for {city.title()}: {output_file}")

compute_utci("monterrey")
compute_utci("amsterdam")
compute_utci("singapore")


UTCI results saved for Monterrey: monterrey_utci_results.csv
UTCI results saved for Amsterdam: amsterdam_utci_results.csv
UTCI results saved for Singapore: singapore_utci_results.csv


In [28]:
## utci_scores, sperate working hours (8-18), and calculate avrg, dropdown of graphs

In [29]:
import pandas as pd
# UTCI score
def classify_utci(utci_value):
    if utci_value <= 26:
        return "1"  # None
    elif utci_value <= 32:
        return "2"  # Moderate
    elif utci_value <= 38:
        return "3"  # Strong
    elif utci_value <= 46:
        return "4"  # Very Strong
    else:
        return "5"  # Extreme

def add_heat_risk_score(city):
    input_file = f"{city}_utci_results.csv"
    output_file = f"{city}_utci_with_measures.csv"

    df = pd.read_csv(input_file)
    df['heat_risk_score'] = df['utci'].apply(classify_utci)
    df.to_csv(output_file, index=False)

    print(f" Heat risk scores added: {output_file}")

for city in ['monterrey', 'amsterdam', 'singapore']:
    add_heat_risk_score(city)


 Heat risk scores added: monterrey_utci_with_measures.csv
 Heat risk scores added: amsterdam_utci_with_measures.csv
 Heat risk scores added: singapore_utci_with_measures.csv


In [30]:
import pandas as pd

def label_shift(city):
    file = f"{city}_utci_with_measures.csv"
    df = pd.read_csv(file)

    df['datetime'] = pd.to_datetime(df['datetime'])
    df['shift'] = df['datetime'].apply(lambda dt: 1 if 8 <= dt.hour < 18 else 0)

    df.to_csv(file, index=False)
    print(f"Shift labels added for {city.title()}: {file}")
    return df.head()

for city in ['monterrey', 'amsterdam', 'singapore']:
    label_shift(city)


Shift labels added for Monterrey: monterrey_utci_with_measures.csv
Shift labels added for Amsterdam: amsterdam_utci_with_measures.csv
Shift labels added for Singapore: singapore_utci_with_measures.csv


In [31]:
import pandas as pd

def merge_with_measures(city):
    utci_file = f"{city}_utci_with_measures.csv"
    measures_file = "heat_risk_database.csv"

    df = pd.read_csv(utci_file)
    df_measures = pd.read_csv(measures_file)

    # Ensure score is integer for merge
    df['heat_risk_score'] = df['heat_risk_score'].astype(int)

    merged_df = pd.merge(df, df_measures, left_on='heat_risk_score', right_on='Heat_Risk_Score', how='left')

    merged_df.to_csv(utci_file, index=False)
    print(f"Measures merged for {city.title()}: {utci_file}")

merge_with_measures("monterrey")
merge_with_measures("amsterdam")
merge_with_measures("singapore")


Measures merged for Monterrey: monterrey_utci_with_measures.csv
Measures merged for Amsterdam: amsterdam_utci_with_measures.csv
Measures merged for Singapore: singapore_utci_with_measures.csv
